In [3]:
import Pkg
Pkg.add("LightGraphs")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


# Graphs Example

## Connected Components

In [1]:
using LightGraphs  # For connectivity checks in the callback

z_vals = [0 1 0; 1 0 0; 0 0 0]

# Check for connectivity using a graph representation
g = SimpleGraph(3)
for i in 1:3
    for i_prime in i+1:3
        if z_vals[i, i_prime] > 0.5
            add_edge!(g, i, i_prime)
        end
    end
end

# Find connected components in the graph
components = connected_components(g)

2-element Vector{Vector{Int64}}:
 [1, 2]
 [3]

## Doubly connected components

In [4]:
using LightGraphs

function merge_sets_union_find(set_of_sets::Set{Set{T}}) where T
    # Step 1: Map each element to a unique subset
    element_to_set = Dict{T, Set{T}}()
    for subset in set_of_sets
        for elem in subset
            element_to_set[elem] = subset
        end
    end

    # Step 2: Union subsets with overlapping elements
    for subset in set_of_sets
        # Find all subsets connected to the current subset
        merged_subset = Set(subset)
        for elem in subset
            merged_subset = union(merged_subset, element_to_set[elem])
        end

        # Update the mapping for all elements in the merged subset
        for elem in merged_subset
            element_to_set[elem] = merged_subset
        end
    end

    # Step 3: Collect unique subsets
    return Set(values(element_to_set))
end


merge_sets_union_find (generic function with 1 method)

In [5]:
function add_strays(graph::AbstractGraph, sets_of_nodes::Set{Set{Int}})
    # Collect all nodes in the given sets
    nodes_in_sets = reduce(union, sets_of_nodes, init=Set{Int}())
    
    # Get all nodes in the graph
    all_nodes = Set(1:nv(graph))
    
    # Find nodes not in any of the sets (strays)
    stray_nodes = setdiff(all_nodes, nodes_in_sets)
    
    # Add each stray node as its own set
    for stray in stray_nodes
        push!(sets_of_nodes, Set([stray]))
    end
    
    return sets_of_nodes
end

add_strays (generic function with 1 method)

In [6]:
function doubly_connected_components(graph)
    cycles = Set(Set(x) for x in filter(x->length(x)>2, simplecycles(DiGraph(graph))))
    doubly_connected_components_no_strays = merge_sets_union_find(cycles)
    return add_strays(graph, doubly_connected_components_no_strays)
end

doubly_connected_components (generic function with 1 method)

In [7]:
# Create a graph
graph = SimpleGraph(7)
add_edge!(graph, 1, 2)
add_edge!(graph, 2, 4)
add_edge!(graph, 4, 3)
add_edge!(graph, 3, 1)
add_edge!(graph, 2, 3)

add_edge!(graph, 4, 5)
add_edge!(graph, 5, 6)
add_edge!(graph, 6, 7)


doubly_connected_components(graph)


Set{Set{Int64}} with 4 elements:
  Set([5])
  Set([7])
  Set([4, 2, 3, 1])
  Set([6])

# Gurobi Callbacks

In [13]:
import JuMP.MathOptInterface

In [12]:
import MathOptInterface as MOI

In [15]:
using JuMP
using Gurobi
using LightGraphs  # For connectivity checks in the callback
using LinearAlgebra  # For distance calculations
import MathOptInterface as MOI  # Import MathOptInterface explicitly

# Example data (modify as needed)
n_facilities = 5  # Number of facilities
n_customers = 10  # Number of customers
k = 3  # Maximum number of facilities that can be selected

# Randomly generate coordinates for facilities and customers
facility_coords = rand(0:100, n_facilities, 2)  # Facility coordinates (x, y)
customer_coords = rand(0:100, n_customers, 2)  # Customer coordinates (x, y)

# Compute distances dynamically
function euclidean_distance(coord1, coord2)
    return norm(coord1 - coord2)
end

# Facility-to-customer distances
d = [euclidean_distance(facility_coords[i, :], customer_coords[j, :]) for i in 1:n_facilities, j in 1:n_customers]

# Facility-to-facility distances
D = [i == i_prime ? 0 : euclidean_distance(facility_coords[i, :], facility_coords[i_prime, :]) for i in 1:n_facilities, i_prime in 1:n_facilities]

# Cost parameters
C = rand(100:200, n_facilities)  # Facility setup costs
w = 5  # Cost per unit length of wire
F = 10  # Cost per unit length of fiber optic cable

# Initialize model
model = Model(Gurobi.Optimizer)

# Decision variables
@variable(model, x[1:n_facilities], Bin)  # Facility setup decision
@variable(model, 0 <= y[1:n_facilities, 1:n_customers] <= 1)  # Fractional connections
@variable(model, z[i=1:n_facilities, i_prime=i+1:n_facilities], Bin)  # Facility-to-facility connections only for i < i_prime

# Objective function: Minimize total costs
@objective(model, Min, sum(C[i] * x[i] for i in 1:n_facilities) +
                          sum(w * d[i, j] * y[i, j] for i in 1:n_facilities, j in 1:n_customers) +
                          sum(F * D[i, i_prime] * z[i, i_prime] for i in 1:n_facilities, i_prime in i+1:n_facilities))

# Constraints
# 1. Customer demand satisfaction
@constraint(model, [j=1:n_customers], sum(y[i, j] for i in 1:n_facilities) == 1)

# 2. Wires only for open facilities
@constraint(model, [i=1:n_facilities, j=1:n_customers], y[i, j] <= x[i])

# 3. Fiber optic cables only for open facilities
@constraint(model, [i=1:n_facilities, i_prime=i+1:n_facilities], z[i, i_prime] <= x[i])
@constraint(model, [i=1:n_facilities, i_prime=i+1:n_facilities], z[i, i_prime] <= x[i_prime])

# 4. Limit the number of selected facilities
@constraint(model, sum(x[i] for i in 1:n_facilities) <= k)

# 5. Connectivity callback to enforce at least 2 cables between partitions
function connectivity_callback(context::MathOptInterface.CallbackContext)
    # Get the values of the x and z variables
    x_vals = MOI.get(context, MOI.CallbackVariablePrimal, x)
    z_vals = Dict((i, i_prime) => MOI.get(context, MOI.CallbackVariablePrimal, z[i, i_prime]) for i in 1:n_facilities for i_prime in i+1:n_facilities)

    # Identify selected facilities
    selected_facilities = [i for i in 1:n_facilities if x_vals[i] > 0.5]

    # Skip if fewer than 2 facilities are selected
    if length(selected_facilities) <= 1
        return
    end

    # Check for connectivity using a graph representation
    g = SimpleGraph(n_facilities)
    for i in 1:n_facilities
        for i_prime in i+1:n_facilities
            if z_vals[(i, i_prime)] > 0.5
                add_edge!(g, i, i_prime)
            end
        end
    end

    # Find connected components in the graph
    components = connected_components(g)

    # For each disconnected subset, enforce the connectivity constraint
    for component in components
        complement = setdiff(1:n_facilities, component)
        if !isempty(component) && !isempty(complement)
            MOI.submit(context, MOI.LazyConstraint,
                sum(z[min(i, i_prime), max(i, i_prime)] for i in component, i_prime in complement) >= 2)
        end
    end
end

# Attach the lazy constraint callback
MOI.set(model, MOI.LazyConstraintCallback(), connectivity_callback)

# Solve the model
optimize!(model)

# Output results
println("Optimal Objective Value: ", objective_value(model))
println("Facilities to open: ", [i for i in 1:n_facilities if value(x[i]) > 0.5])
println("Customer-facility connections (y):")
for i in 1:n_facilities, j in 1:n_customers
    if value(y[i, j]) > 1e-6  # Small threshold to filter non-zero values
        println("  Customer $j -> Facility $i: ", value(y[i, j]))
    end
end
println("Facility-facility connections (z):")
for i in 1:n_facilities, i_prime in i+1:n_facilities
    if value(z[i, i_prime]) > 0.5
        println("  Facility $i <-> Facility $i_prime")
    end
end


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-18


LoadError: UndefVarError: `MathOptInterface` not defined

In [18]:
using JuMP, Gurobi, Test

model = direct_model(Gurobi.Optimizer())
@variable(model, 0 <= x <= 2.5, Int)
@variable(model, 0 <= y <= 2.5, Int)
@objective(model, Max, y)
cb_calls = Cint[]
function my_callback_function(cb_data, cb_where::Cint)
    # You can reference variables outside the function as normal
    push!(cb_calls, cb_where)
    # You can select where the callback is run
    if cb_where != GRB_CB_MIPSOL && cb_where != GRB_CB_MIPNODE
        return
    end
    # You can query a callback attribute using GRBcbget
    if cb_where == GRB_CB_MIPNODE
        resultP = Ref{Cint}()
        GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_STATUS, resultP)
        if resultP[] != GRB_OPTIMAL
            return  # Solution is something other than optimal.
        end
    end
    # Before querying `callback_value`, you must call:
    Gurobi.load_callback_variable_primal(cb_data, cb_where)
    x_val = callback_value(cb_data, x)
    y_val = callback_value(cb_data, y)
    # You can submit solver-independent MathOptInterface attributes such as
    # lazy constraints, user-cuts, and heuristic solutions.
    if y_val - x_val > 1 + 1e-6
        con = @build_constraint(y - x <= 1)
        MOI.submit(model, MOI.LazyConstraint(cb_data), con)
    elseif y_val + x_val > 3 + 1e-6
        con = @build_constraint(y + x <= 3)
        MOI.submit(model, MOI.LazyConstraint(cb_data), con)
    end
    return
end
# You _must_ set this parameter if using lazy constraints.
MOI.set(model, MOI.RawOptimizerAttribute("LazyConstraints"), 1)
MOI.set(model, Gurobi.CallbackFunction(), my_callback_function)
optimize!(model)
@test termination_status(model) == MOI.OPTIMAL
@test primal_status(model) == MOI.FEASIBLE_POINT
@test value(x) == 1
@test value(y) == 2

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-18
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x1cb4e750
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+00, 2e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 2 columns, 0 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 2.000000e+00, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       2.0

Test Passed

In [24]:
using JuMP
using Gurobi

# Define the model
model = direct_model(Gurobi.Optimizer())

# Decision variables
@variable(model, 0 <= x <= 2.5, Int)
@variable(model, 0 <= y <= 2.5, Int)

# Objective function
@objective(model, Max, y)

# Callback function
function my_callback_function(cb_data, cb_where::Cint)
    # Run only during solution callbacks
    if cb_where != GRB_CB_MIPSOL
        return
    end

    # Load the solution values for the current node
    Gurobi.load_callback_variable_primal(cb_data, cb_where)
    x_val = callback_value(cb_data, x)
    y_val = callback_value(cb_data, y)

    println("Current solution: x = $x_val, y = $y_val")

    # Add a lazy constraint if x + y > 3
    if y_val - x_val > 1 + 1e-6
        println("CALLBACK!!!!!")
        con = @build_constraint(y - x <= 1)
        MOI.submit(model, MOI.LazyConstraint(cb_data), con)
    end
end

# Enable lazy constraints in Gurobi
MOI.set(model, MOI.RawOptimizerAttribute("LazyConstraints"), 1)

# Attach the callback
MOI.set(model, Gurobi.CallbackFunction(), my_callback_function)

# Solve the model
optimize!(model)

# Print the results
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution:")
    println("Objective value: ", objective_value(model))
    println("x = ", value(x))
    println("y = ", value(y))
else
    println("No optimal solution found.")
end


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-18
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x1cb4e750
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+00, 2e+00]
  RHS range        [0e+00, 0e+00]
Current solution: x = -0.0, y = 2.0
CALLBACK!!!!!
Current solution: x = 2.0, y = 2.0
Found heuristic solution: objective 2.0000000
Presolve time: 0.00s
Presolved: 0 rows, 2 columns, 0 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 2 

O